In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
from langchain import llms

In [2]:
index_creator = VectorstoreIndexCreator()

In [3]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../state_of_the_union.txt')
docsearch = index_creator.from_loaders([loader]).vectorstore

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [4]:
from langchain.chat.chat_vector_db import ChatVectorDBChain
from langchain.chat_models.openai import OpenAI

## Memory outside the chain

In [5]:
chain = ChatVectorDBChain.from_llm(model = OpenAI(temperature=0), llm=llms.OpenAI(temperature=0), vectorstore=docsearch)

In [6]:
from langchain.memory.chat_memory import ChatMemory

In [7]:
memory = ChatMemory()

In [8]:
query = "What did the president say about Ketanji Brown Jackson"
answer = chain.run(question=query, chat_history=memory.messages)
answer

"The President said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson for the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. He also mentioned that she has received broad support from various groups, including the Fraternal Order of Police and former judges appointed by Democrats and Republicans."

In [9]:
memory.add_user_message(query)
memory.add_ai_message(answer)

In [10]:
query = "Did the president say who she suceeded"
chain.run(question=query, chat_history=memory.messages)

'Ketanji Brown Jackson has not yet been confirmed as a United States Supreme Court Justice. She has been nominated by President Biden to succeed Justice Stephen Breyer, who is retiring.'

In [11]:
answer

"The President said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson for the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence. He also mentioned that she has received broad support from various groups, including the Fraternal Order of Police and former judges appointed by Democrats and Republicans."

## Memory in the chain

In [5]:
from langchain.chat.memory import ChatHistoryMemory
chain = ChatVectorDBChain.from_llm(
    model = OpenAI(temperature=0), 
    llm=llms.OpenAI(temperature=0), 
    vectorstore=docsearch,
    memory=ChatHistoryMemory(input_key="question")
)

In [6]:
query = "What did the president say about Ketanji Brown Jackson"
answer = chain.run(question=query)
answer

'The President said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson for the United States Supreme Court, and that she is one of our nation’s top legal minds who will continue Justice Breyer’s legacy of excellence. He also mentioned that she is a former top litigator in private practice, a former federal public defender, and comes from a family of public school educators and police officers.'

In [7]:
query = "Did the president say who she suceeded"
chain.run(question=query)

'Ketanji Brown Jackson has not yet been confirmed as a United States Supreme Court Justice. She has been nominated by President Biden to succeed Justice Stephen Breyer, who is retiring.'